# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-06 00:00:00  to  2020-04-07 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,115,119,4,3.478261
1,02,223,255,32,14.349776
2,03,50,52,2,4.000000
3,04,35,29,-6,-17.142857
4,05,63,64,1,1.587302


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,3119,3169,50,1.603078
94,94,1811,1956,145,8.006626
92,92,1978,1932,-46,-2.325581
93,93,1406,1461,55,3.911807
69,69,1223,1174,-49,-4.006541


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -95  Mean:  3.3333333333333335  Max:  145


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -25.0  Mean %:  0.4242871227971443  Max %:  23.809523809523807


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
3,04,35,29,-6,-17.142857
11,12,59,55,-4,-6.779661
12,13,1169,1074,-95,-8.126604
14,15,28,26,-2,-7.142857
15,16,15,12,-3,-20.000000
17,18,55,54,-1,-1.818182
18,19,44,41,-3,-6.818182
20,22,75,69,-6,-8.000000
22,24,28,26,-2,-7.142857
24,26,260,244,-16,-6.153846


In [11]:
print(stats.query("diff < 0").shape)

(42, 5)


#### <br> Departments with highest evolution of hospitalizations (total number)

In [12]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
94,94,1811,1956,145,8.006626
93,93,1406,1461,55,3.911807
75,75,3119,3169,50,1.603078
77,77,692,740,48,6.936416
62,62,391,431,40,10.230179


#### <br> Departments with highest rate of hospitalizations (%)

In [13]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
40,40,21,26,5,23.809524
21,23,11,13,2,18.181818
39,39,77,90,13,16.883117
1,02,223,255,32,14.349776
79,79,28,32,4,14.285714


#### <br> Focus on some departments

In [14]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,3119,3169,50,1.603078


In [15]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,193,191,-2,-1.036269


In [16]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,235,230,-5,-2.12766
